In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import json
import os
from openai import OpenAI
import logging
import json
import numpy as np
from transformers import CLIPProcessor, CLIPModel

In [5]:
with open ('electronics.json', 'r') as file:
    data = json.load(file)

In [6]:
for phone in data['products']['phones'][:3]:
    print(phone)

{'brand': 'Apple', 'model': 'iPhone 11 Pro Max', 'color': 'Gray', 'storage': '128 GB', 'condition': 'New', 'price': 1233.33, 'city': 'Warsaw'}
{'brand': 'Samsung', 'model': 'Galaxy S21', 'color': 'Blue', 'storage': '64 GB', 'condition': 'Used', 'price': 740.0, 'city': 'Krakow'}
{'brand': 'Xiaomi', 'model': 'Mi 11', 'color': 'Black', 'storage': '256 GB', 'condition': 'New', 'price': 999.99, 'city': 'Wroclaw'}


In [16]:
conda init

no change     /opt/anaconda3/condabin/conda
no change     /opt/anaconda3/bin/conda
no change     /opt/anaconda3/bin/conda-env
no change     /opt/anaconda3/bin/activate
no change     /opt/anaconda3/bin/deactivate
no change     /opt/anaconda3/etc/profile.d/conda.sh
no change     /opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /opt/anaconda3/shell/condabin/Conda.psm1
no change     /opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /opt/anaconda3/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /opt/anaconda3/etc/profile.d/conda.csh
no change     /Users/dzmitry.ryzhov/.bash_profile
No action taken.

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
os.system('conda activate myenv')


CondaError: Run 'conda init' before 'conda activate'



256

In [30]:
OPENAI_API_KEY = os.getenv("MY_API_KEY")

In [8]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [112]:
def generate_description(category, product):
    prompt = f"""
Generate a concise, engaging product description in two sentences, highlighting distinct features and unique usage details.
Include relevant condition insights, such as battery health for USED phones OR years of use for other eletronic devices. If a phone condition is NEW then do not inculde any information about battery health.
Example below:
Iphone 11 Pro with 128 GB storage, meticulously cared for in a silicone case and glass protector, showing only minor screen wear. Battery health is at 80%, reflecting typical use over four years, and it comes with a charging cable and screen protector in the box.
Let's get started, the item below:
Product Type= {category}\n
Brand= {product['brand']}\n
Model= {product['model']}\n
Color= {product.get('color', 'N/A')}\n
Storage= {product.get('storage', 'N/A')}\n
Type= {product.get('type', 'N/A')}\n
Size= {product.get('size', 'N/A')}\n
Resolution= {product.get('resolution', 'N/A')}\n
Condition= {product['condition']}\n
Price= {product['price']}\n
City= {product['city']}\n
Description: """.format(
        category=category,
        brand=product['brand'],
        model=product['model'],
        color=product.get('color', ''),
        storage=product.get('storage', ''),
        type=product.get('type', ''),
        size=product.get('size', ''),
        resolution=product.get('resolution', ''),
        condition=product['condition'],
        price=product['price'],
        city=product['city']
    ).strip()
    for attempt in range(3): 
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0.2
            )
            description = response.choices[0].message.content.strip()
            return description
        except Exception as e:
            logging.error(f"Error generating description for {product['brand']} {product['model']}: {e}")
            return "Description not available."

In [113]:
for category, products in data['products'].items():
    for product in products:
        product['description'] = generate_description(category, product)
    print(f'{category} are done')

phones are done
tv are done
coffee_machines are done
tablets are done


In [124]:
id = 1
for category, product in data['products'].items():
    for item in product:
        item['id'] = id
        id += 1

In [125]:
products['products']['phones'][:2]

[{'brand': 'Apple',
  'model': 'iPhone 11 Pro Max',
  'color': 'Gray',
  'storage': '128 GB',
  'condition': 'New',
  'price': 1233.33,
  'city': 'Warsaw',
  'description': "Introducing the brand-new iPhone 11 Pro Max in sleek Gray, featuring a generous 128 GB of storage for all your apps, photos, and videos. Experience cutting-edge technology with its stunning display and powerful performance, all backed by Apple's renowned quality and support, available now in Warsaw for just 1233.33.",
  'id': 1},
 {'brand': 'Samsung',
  'model': 'Galaxy S21',
  'color': 'Blue',
  'storage': '64 GB',
  'condition': 'Used',
  'price': 740.0,
  'city': 'Krakow',
  'description': 'Samsung Galaxy S21 in a stunning blue finish, featuring 64 GB of storage and a sleek design that fits comfortably in your hand. This gently used device shows minimal signs of wear and has a battery health of 85%, ensuring reliable performance for everyday tasks, and comes with a charging cable included.',
  'id': 2}]

In [2]:
with open ('updated_info.json', 'w') as file:
    json.dump(data, file, indent = 4)

NameError: name 'data' is not defined

In [4]:
with open ('updated_info.json', 'r') as items:
    electronic_products = json.load(items)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [128]:
electronic_products['products']['phones'][:1]

[{'brand': 'Apple',
  'model': 'iPhone 11 Pro Max',
  'color': 'Gray',
  'storage': '128 GB',
  'condition': 'New',
  'price': 1233.33,
  'city': 'Warsaw',
  'description': "Introducing the brand-new iPhone 11 Pro Max in sleek Gray, featuring a generous 128 GB of storage for all your apps, photos, and videos. Experience cutting-edge technology with its stunning display and powerful performance, all backed by Apple's renowned quality and support, available now in Warsaw for just $1233.33.",
  'id': 1}]

In [147]:
pip freeze

annotated-types==0.7.0
anyio @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_68kdsx8iyd/croot/anyio_1729121281958/work
appnope @ file:///Users/ktietz/demo/mc3/conda-bld/appnope_1629146036738/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/croot-wbf5edig/argon2-cffi-bindings_1644845754377/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-lru @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_02efro5ps8/croot/async-lru_1699554529181/work
attrs @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_73lbj1xbpc/croot/attrs_1729089415510/work
Babel @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_00k1rl2pus/croot/babel_1671781944131/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt